In [ ]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
import wandb
import importlib
import utils

importlib.reload(utils)
from utils import *

## Parameters

### Initial values

In [2]:
# initial value

GWP_0 = 8.5e13 #initial world GDP, used by takeoff model
RH_0=1.7e11 #initial research input to hardware (only relevant for exogeneous modelling)
RS_0=1.7e10 #initial research input to software (only releveant for exogeneous modelling)
HS_0 =  2e28 #https://docs.google.com/document/d/1rw1pTbLi2brrEP0DcsZMAVhlKp6TKGKNUSFRkkdP_hs/edit#bookmark=id.b15qyylbeqxp
K_0= 6.1e14 #cumulative sum of depreciation adjusted capital investments
L_0 = 4e9
QH_0=2.2e9 #cumulative hardware research (USD) at start of simulation, read from graph (unsure how this is determined)
QS_0=1e8 #at start of simulation, no software research done
SK_0=0.5 #initial capital factor share
SCog_0=0.5 #initial cognitive factor share
SC_0=0.01 #initial factor share of compute in cognitive output production
SL_0=0.99  #initial factor share of labour in cognitive output production


S_0=1 
H_0=1.5e17 #needs to be updated
C_eff_0=HS_0*S_0 #initial stocks of effective compute



#### General

In [ ]:
HS_depreciation_rate = 0.2 #hardware stock deprecitation rate
s_K = 0.2 #savings rate - 0.2/year
f_GWP_H = 8e-4 #fraction of world output spent on hardware
TFP = 1 #Total factor productivity - grows exogeneously in FTM (not implemented yet)
T = 30 #number of years to run simulation for
delta_T=1 #size of time increment. n_steps=T/delta_T
growth_exponent = 1/delta_T #so we know which root to take of growth rates
n_steps = T/delta_T

g_L = 0.01

#### Hardware research

In [ ]:
#### hardware research parameters 
alpha_H = 0.0092 # weight for capital in hardware research production (used when shares are NOT computed)
KS_H=None #capital share in hardware research productions
CogS_H=None #cognitive share in hardware production
f_K_H = 2e-3 # 2e-3 #fraction of capital stock for hardware research production
f_L_H = 2e-3 #2e-3 #fraction of labour stock for hardware research production
f_C_H = 2e-3
f_GWP_H=2e-3 #t=0 fraction of GWP spent on hardware research (~ $140B on semiconductor R&D in 2022)
rho_H = 1 #substitution parameter between capital and cogntiive labour for hardware research production
rH = 7.4286 #hardware efficiency returns to hardware research - determined from comparing historical data on hardware (H) growth and research input ($) growth
lambda_H = 0.7 #'stepping on toes' effect for hardware research (i.e: How much research per unit time contributes to hardware stock)
ratio_initial_cumulative_hardware_input = 0.063 #ratio of 2022 hardware input to cumulative hardware input (in $)


#### Software research

In [ ]:
#### software research parameters 
alpha_S = 0.2571 #weight for capital in software research production
KS_S=None #capital share in software research productions
CogS_S=None #cognitive share in software research production
f_K_S = 2e-4 #2e-4 fraction of capital stock for software research production
f_L_S = 2e-4 #2e-4 #fraciton of labour stock for software research production
f_C_S = 2e-4
f_GWP_S = 2e-4 #only needed for t=0. Payments to software researchers estimated as ~$10-20b, which is approx. 0.02% of GWP
rho_S = 1 #substitution parameter between capital and cognitive labour for software research production
rS = 1.7857
ratio_initial_cumulative_software_input = 0.2 #ratio of initial software inputs to cumulative software research inputs


#### GWP production

In [ ]:
#### GWP production
alpha_G = 0.0002 #task weight non-compute capital in GWP production - should be computed each iteration
KS_G=None #capital share in goods productions
CogS_G=None #cognitive share in goods production
f_K_GWP = 1-f_K_H-f_K_S #fraction of capital stock for producing GWP - not modelling hardware and software research endogeneously right now
f_L_GWP = 1-f_L_H-f_L_S #fraction of labour for producing GWP - not modelling hardware and software research endogeneously right now
f_C_GWP = 1-f_C_H-f_C_S #fraction of effective compute for producing GWP - not modelling hardware and software research endogeneously right now
rho_G = -0.4 #substitution parameter between capital and cognitive labour for GWP production
psi_G = -0.5 #substitution parameter between cognitive tasks in cognitive input production



#### Automation

In [ ]:
#automation - training
n_labour_tasks = 100
f_C_T = 1.6e-4 #fraction of compute assigned to frontier training run in a year
automation_training_task100 = 36 #compute threshold for last task (logspace)
training_FLOP_GAP=4
automation_training_threshold=construct_automation_thresholds(automation_training_task100,training_FLOP_GAP)

#automation - runtime
eta_0 = 6.0e23
persecond_runtime_100 = 16 + 1 - np.log10(6) #bioanchors is 1e16 FLOP/s, adjust up by 1 OOM to account for TAI v.s. AGI, and down by 6x for AGI one-time benefits compared to human
runtime_100 = persecond_runtime_100 + np.log10(60*60*24*365)
runtime_FLOP_GAP=1
baseline_runtime_requirements = construct_baseline_runtime_efficiencies(runtime_100,runtime_FLOP_GAP)
delta=1.5 # Scaling factor of runtime efficiency C_T less than/greater than runtime requirements




In [ ]:

#### checks
assert f_K_GWP+f_K_H+f_K_S==1
assert f_L_GWP+f_L_H+f_L_S==1
assert f_C_GWP+f_C_H+f_C_S==1

## Main loop

In [ ]:

#logging setup
LOGGING=False
logging_dict = {
    't':None,
    'year':None,

    'GWP (2022 $)':None,
    'HS (FLOP/year)':None,
    'C_eff (2022 FLOP/year)':None,
    'H (FLOP/$/year)':None,
    'S (2022 FLOP/FLOP/year)':None,
    'RS (2022 $)':None,

    'Cog_G (2022 $)':None, #still no idea how to physically interpret this variable
    'Automation index':None,
    'C_eff_T':None,

    'K (2022 $)':None,
    'L (HLY/year)':None, #HLH - human labour years
    'Capital to cognitive share ratio':None,
    'alpha_G':None,
    'Compute to labour share ratio':None,
    'beta_0':None,
    'beta_i':None,

    'RH (2022 $)':None,
    'QH (2022 $)':None,
    'g_H':None,
    'g_QH':None,

    'RS (2022 $)':None,
    'QS (2022 $)':None,
    'g_S':None,
    'g_QS':None,
}



#### bools

##production bools
GWP_ENDOGENEOUS,exg_g_GWP=True,0.02
H_ENDOGENEOUS,g_H=False,0.5 #use derivate method of Jones 1995 research modeling, if not hardware (H) exogeneous
S_ENDOGENEOUS,g_S=False,0.5 #use derivate method of Jones 1995 research modeling, if not software (S) exogeneous
RH_ENDOGENEOUS,ex_g_RH=False,0.02 #produce hardware research each time step, if not hardware production (RH) exogeneous
RS_ENDOGENEOUS,ex_g_RS=False,0.02 #produce software research each time step, if not  software production (RS) exogeneous
COGNITIVES_INPUT_ENDOGENOUS,g_Cog=True,0.02 #'Produce' (calculate) cognitive inputs each time step, if not cognitive inputs (Cog) exogeneous
COMPUTE_OUTER_WEIGHTS,alpha_G=True,0.0002 #compute outer weights to CES production function, if not constant
COMPUTE_INNER_WEIGHTS,beta_0,beta_i = True,1,4e-11 #compute inner weights to CES production function, if not constant
COMPUTE_RUNTIME_EFFICIENCIES = True

##Research bools
HARDWARE_PENALTY_FACTOR=False #apply penalty factor to hardware efficiency growth rate - leave for now
SOFTWARE_PENALTY_FACTOR=False #apply penalty factor to software efficiency growth rate - leave for now

automation_index=0

### main loop
if LOGGING:
    run=wandb.init(project="FTM-implementation")


for t in range (int(n_steps)):
    year=2022+t

    if t==0: #### initial state variables

        GWP = GWP_0 
        HS =  HS_0 
        C_eff=C_eff_0 #initial stock of effective compute
        S=S_0
        H=H_0

        Cog_G=None
        automation_index=None
        C_T=f_C_T*C_eff

        K = K_0 
        L = L_0

        RH_0=f_GWP_H*GWP_0
        RH=RH_0
        QH_0=(RH_0**lambda_H)/ratio_initial_cumulative_hardware_input ##FTM also multiplies by 1/lambda_H; unsure why
        QH=QH_0
        g_QH=None 

        RS_0=f_GWP_S*GWP_0
        RS=RS_0
        QS_0=(RS**lambda_H)/ratio_initial_cumulative_software_input ##FTM also multiplies by 1/lambda_H; unsure why
        QS=QS_0
        g_QS=None

    else: #### update factors of production

        #### non output variables
        g_K=(s_K*GWP)/K
        K=(1+g_K)*K #depreciation not implemented yet

        #L 
        L = (1+g_L)*L
    

    if 1: #### ALLOCATIONS (static for now - FTM has these varying over time)

        #allocating compute
        C_eff_GWP=f_C_GWP*C_eff #effective compute in GWP production

        #allocating labour
        L_GWP=f_L_GWP*L
        L_H=f_L_H*L
        L_S=f_L_S*L

        #allocating capital
        K_GWP=f_K_GWP*K
        K_H=f_K_H*K
        K_S=f_K_S*K

        #allocating compute 
        C_eff_GWP=f_C_GWP*C_eff
        C_eff_H=f_C_H*C_eff
        C_eff_S=f_C_S*C_eff
        C_T= f_C_T*C_eff
        
    
    if 1: #### set automation conditions (not differentiating between GWP and research for now)

        C_T = f_C_T * C_eff #largest training run in this timestep (does not account for training run time)

        ####set automation index
        automation_index = set_automation_index(automation_training_threshold,C_T,n_labour_tasks)


    if GWP_ENDOGENEOUS: #production and GWP

        if COGNITIVES_INPUT_ENDOGENOUS:

            labour_array,effective_compute_array = construct_labour_compute_arrays(L_GWP,C_eff_GWP,n_labour_tasks,automation_index,optimal_allocation=False)

            if COMPUTE_RUNTIME_EFFICIENCIES:
                    eta = compute_runtime_efficiencies(baseline_runtime_requirements,C_T,automation_training_threshold,delta)
            else:
                    eta = np.ones(n_labour_tasks)*eta_0



            #inner weights
            if COMPUTE_INNER_WEIGHTS:
                if t==0:
                    compute_to_labour_ratio=SC_0/SL_0
                else:
                    compute_to_labour_ratio = (beta_0/(n_labour_tasks*beta_i))*((C_eff_GWP*n_labour_tasks)/L_GWP)**(psi_G)

                inner_weights = compute_inner_weights(compute_to_labour_ratio,L_GWP,C_eff_GWP,n_labour_tasks,psi_G)

            else:
                inner_weights = np.concatenate(\
                    (np.ones(1)*beta_0,np.ones(n_labour_tasks)*beta_i),\
                    axis=0
                    )
            
            task_inputs = compute_task_inputs(labour_array,effective_compute_array,eta)
            Cog_G = (np.sum(inner_weights*(task_inputs)**(psi_G)))**(1/psi_G) #compute->labour efficiencies NOT YET IMPLEMENTED
        else:
            Cog_G = 2.0e24 #constant if not endogenised

        if COMPUTE_OUTER_WEIGHTS:
            if t==0:
                capital_to_cognitive_ratio=SK_0/SCog_0
            else:
                capital_to_cognitive_ratio = (alpha_G/(1-alpha_G))*((K_GWP/Cog_G)**rho_G)
            
            alpha_G = compute_outer_weights(capital_to_cognitive_ratio,K_GWP,Cog_G,rho_G)

        else:
            alpha_G=alpha_G

        
        production_gwp = compute_production(alpha=alpha_G,K=K_GWP,Cog=Cog_G,rho=rho_G,TFP=1)

        if t==0: #all we need is gwp_to_production ratio
            gwp_to_production_ratio = GWP/production_gwp
        else:
            GWP=gwp_to_production_ratio * production_gwp

    else:
        if t==0: pass
        else: GWP=(1+exg_g_GWP)*GWP #change GWP exogeneously


    if H_ENDOGENEOUS:
        
                #hardware/software research production (only relevant if )
        if RH_ENDOGENEOUS:
            
            if COGNITIVES_INPUT_ENDOGENOUS:

                labour_per_task = L_H/n_labour_tasks #divide labour force for hardware research uniformly between tasks (not doing optimal allocation right now)
                effective_compute_per_task = C_eff_H/n_labour_tasks
                
                #labour and compute arrays
                labour_array = np.concatenate(\
                    (np.zeros(1),np.ones(n_labour_tasks) * labour_per_task),\
                    axis=0
                    )
                effective_compute_array = np.concatenate( \
                    (np.ones(automation_index+1)*effective_compute_per_task,np.zeros(n_labour_tasks-automation_index)),\
                    axis=0
                    )

            else:
                Cog_H = 4.0e21 # but what does 'cognitive input' mean?

            if COMPUTE_OUTER_WEIGHTS:
                if t==0:
                    pass
                else:
                    pass
            else:
                alpha_H=alpha_H

            production_hardware = TFP * (alpha_H*K_H**rho_H + (1-alpha_H)*Cog_H**rho_H)**(1/rho_H)

            if t==0:
                RH_to_production_ratio  = RH/production_hardware
            else:
                RH = RH * production_hardware

            if COMPUTE_OUTER_WEIGHTS: #Not implemented yet
                pass
            else:
                alpha_H=alpha_H
            
            production_hardware = TFP * (alpha_H*K_H**rho_H + (1-alpha_H)*Cog_H**rho_H)**(1/rho_H)

            if t==1:
                hardware_to_production_ratio = RH_0/production_hardware

            RH = hardware_to_production_ratio * production_hardware
                
        else: RH=(1+ex_g_RH)*RH

        g_QH=(delta_T*(RH**lambda_H))/QH
        QH=(1+g_QH)*QH

        if HARDWARE_PENALTY_FACTOR:
            pass
        else:
            g_H=rH*g_QH

    else:

        if t==0: pass
        else:
            g_QH=None #no cumulative hardware research required
            H=(1+g_H)*H


    if S_ENDOGENEOUS:

        if RS_ENDOGENEOUS:

            if COMPUTE_OUTER_WEIGHTS:
                pass #not implemented
            else:
                alpha_S=alpha_S
            
            if COGNITIVES_INPUT_ENDOGENOUS: #not implemented
                pass
            else:
                Cog_S = 4.0e20 #

            production_software = TFP * (alpha_S*K_S**rho_S + (1-alpha_S)*Cog_S**rho_S)**(1/rho_S)

            if t==1:
                software_to_production_ratio = RS_0/production_software

            RS = software_to_production_ratio * production_software

        else: RS=(1+ex_g_RS)*RS

        g_QS=(delta_T*(RS**lambda_H))/QS #stepping on toes parameter same for hardware and software research (man the abstractness of growth models is beautiful!)
        QS=(1+g_QS)*QS

        if SOFTWARE_PENALTY_FACTOR:
            pass
        else:
            g_S=rS*g_QS

    else:
        if t==0: pass
        else:
            g_QS=None
            S=(1+g_S)*S

    #hardware stock and effective compute update 
    if t==0:
        pass #do nothing on first pass
    else:

        g_HS = (f_GWP_H*GWP*H - HS_depreciation_rate*HS)/HS
        HS = (1+g_HS)*HS; 

        C_eff = HS*S; 

    #logging
    if LOGGING:
        #we do this at the start to log initial values of variable
        logging_dict['t']=t
        logging_dict['year']=year

        logging_dict['GWP (2022 $)']=GWP
        logging_dict['HS (FLOP/year)']=HS
        logging_dict['C_eff (2022 FLOP/year)']=C_eff
        logging_dict['H (FLOP/$/year)']=H
        logging_dict['S (2022 FLOP/FLOP/year)']=S

        logging_dict['Cog_G (2022 $)']=Cog_G
        logging_dict['Automation index']=automation_index
        logging_dict['C_eff_T']=C_T

        logging_dict['K (2022 $)']=K
        logging_dict['L (HLY/year)']=L
        logging_dict['Capital to cognitive share ratio']=capital_to_cognitive_ratio
        logging_dict['alpha_G']=alpha_G
        logging_dict['Compute to labour share ratio']=compute_to_labour_ratio
        logging_dict['beta_0']=beta_0
        logging_dict['beta_i']=beta_i

        logging_dict['RH (2022 $)']=RH
        logging_dict['QH (2022 USD)']=QH
        logging_dict['g_H']=g_H
        logging_dict['g_QH']=g_QH

        logging_dict['RS (2022 $)']=RS
        logging_dict['QS (2022 USD)']=QS
        logging_dict['g_S']=g_S
        logging_dict['g_QS']=g_QS


        run.log(logging_dict)
        
if LOGGING:
    wandb.finish()


print('Finish')